In [13]:
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils
from utils import plots, get_batches, plot_confusion_matrix, get_data


from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
PROJECT_DIR  = '/home/ubuntu/Extracting-food-preferences'
PICTURES_DIR = '/home/ubuntu/Extracting-food-preferences/classification_with_pics'

In [11]:
PROJECT_DIR  = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences'
PICTURES_DIR = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences\\classification_with_pics'

In [9]:
%cd $PROJECT_DIR

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences


In [7]:
#import modules
from utils import *
from vgg16 import Vgg16
from datetime import datetime

Using Theano backend.


In [ ]:
%cd $PICTURES_DIR

#Set path to sample/ path if desired
path = PICTURES_DIR #+ '/sample'
#path = PICTURES_DIR + '\\' + 'sample\\'
results_path=path + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [28]:
%cd $PICTURES_DIR

#Set path to sample/ path if desired
path = PICTURES_DIR + '\\sample\\'
#path = PICTURES_DIR + '\\' + 'sample\\'
results_path=path + '\\results\\'
train_path=path + '\\train\\'
valid_path=path + '\\valid\\'

model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences\classification_with_pics


In [17]:
#Set constants
batch_size=25
no_of_epochs=1

In [78]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=10)
batches     = get_batches(path+'train', shuffle=False, batch_size=10)

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [125]:
batches.filenames[:10]

['popular\\True.recipe-102411041699034.jpg',
 'popular\\True.recipe-102671041844798.jpg',
 'popular\\True.recipe-1057321210955840.jpg',
 'popular\\True.recipe-1088301214749180.jpg',
 'popular\\True.recipe-1112421217265658.jpg',
 'popular\\True.recipe-1141581220439774.jpg',
 'popular\\True.recipe-1190261224856981.jpg',
 'popular\\True.recipe-119211050673169.jpg',
 'popular\\True.recipe-1195031225279456.jpg',
 'popular\\True.recipe-1250081230025580.jpg']

In [120]:
#import Vgg16 
vgg = Vgg16()
model = vgg.model

In [139]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [128]:
trn_features = vgg.predict_gen(batches)
val_features = vgg.predict_gen(val_batches)

In [138]:
trn_labels.shape

(20, 2)

In [131]:
save_array(model_path+'train_lastlayer_features.bc', trn_features[1])
save_array(model_path+'valid_lastlayer_features.bc', val_features[1])

In [132]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

In [133]:
# 1000 inputs, since that's the saved features, and 2 outputs, for dog and cat
lm = Sequential([ Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [134]:
batch_size=20

In [140]:
lm.fit(trn_features, trn_labels, nb_epoch=5, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/5
200/200 [==============================] - 0s - loss: 0.7219 - acc: 0.5400 - val_loss: 0.7725 - val_acc: 0.4400
Epoch 2/5
200/200 [==============================] - 0s - loss: 0.6346 - acc: 0.6300 - val_loss: 0.7716 - val_acc: 0.4400
Epoch 3/5
200/200 [==============================] - 0s - loss: 0.6086 - acc: 0.6800 - val_loss: 0.8055 - val_acc: 0.4600
Epoch 4/5
200/200 [==============================] - 0s - loss: 0.5720 - acc: 0.7150 - val_loss: 0.8194 - val_acc: 0.4400
Epoch 5/5
200/200 [==============================] - 0s - loss: 0.5654 - acc: 0.7100 - val_loss: 0.8422 - val_acc: 0.5400


In [141]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_26 (Dense)                 (None, 2)             2002        dense_input_2[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________
